# FILTER

In [ ]:
import pandas as pd
from Bio import SeqIO


In [ ]:
input_merged_summary=snakemake.input.merged_summary
input_vOTUs_prefiltered=snakemake.input.vOTUs_prefiltered
input_vibrant_circular=snakemake.input.vibrant_circular
input_vibrant_positive=snakemake.input.vibrant_positive
input_vibrant_quality=snakemake.input.vibrant_quality
input_vibrant_summary=snakemake.input.vibrant_summary
input_virsorter_table=snakemake.input.virsorter_table
input_virsorter_positive_list=snakemake.input.virsorter_positive_list
input_genomad_virus_summary=snakemake.input.genomad_virus_summary
input_genomad_plasmid_summary=snakemake.input.genomad_plasmid_summary
input_genomad_viral_fasta=snakemake.input.genomad_viral_fasta
input_genomad_viral_fasta_conservative=snakemake.input.genomad_viral_fasta_conservative
#________
output_summary=snakemake.output.summary



In [ ]:
checkv_df= pd.read_csv(input_merged_summary, sep="\t",
                            names= ["contig_id_vOTU", "contig_length_vOTU", "provirus_vOTU", "proviral_length_vOTU",
                           "gene_count_vOTU", "viral_genes_vOTU", "host_genes_vOTU", "checkv_quality_vOTU",
                           "miuvig_quality_vOTU", "completeness_vOTU", "completeness_method_vOTU",
                           "contamination_vOTU", "kmer_freq_vOTU", "warnings_vOTU"])
checkv_df

In [ ]:
vOTU_list=[]
for record in SeqIO.parse(input_vOTUs_prefiltered, "fasta"): vOTU_list.append(record.id)
vOTU_prefiltered=pd.DataFrame(vOTU_list, columns=["vOTU_name"])
checkv_df= pd.read_csv(input_merged_summary, sep="\t",
                            names= ["checkV_contig_id", "checkV_contig_length", "checkV_provirus", "checkV_proviral_length",
                           "checkV_gene_count", "checkV_viral_genes", "checkV_host_genes", "checkv_quality",
                           "checkV_miuvig_quality", "checkV_completeness", "checkV_completeness_method",
                           "checkV_contamination", "checkV_kmer_freq", "checkV_warnings"])
vibrant_circular_df=pd.read_csv(input_vibrant_circular, names=["VIBRANT_circular_name"]); vibrant_circular_df["VIBRANT_circular"]="Y"
vibrant_positive_df=pd.read_csv(input_vibrant_positive, names=["VIBRANT_positive_name"]); vibrant_positive_df["VIBRANT_positive"]="Y"
vibrant_quality_df=pd.read_csv(input_vibrant_quality, sep ="\t"); vibrant_quality_df.columns=["VIBRANT_scaffold","VIBRANT_lifecycle","VIBRANT_quality",]; vibrant_quality_df=vibrant_quality_df.groupby("VIBRANT_scaffold").first()
vibrant_summary_df=pd.read_csv(input_vibrant_summary, sep ="\t") ; vibrant_summary_df.columns = ['VIBRANT_' + str(col).replace(" " , "_") for col in vibrant_summary_df.columns]
virsorter_summary_df=pd.read_csv(input_virsorter_table, sep ="\t"); virsorter_summary_df.columns = ['VirSorter2_' + str(col).replace(" " , "_") for col in virsorter_summary_df.columns]; virsorter_summary_df["VirSorter2_positive"]="Y"
genomad_summary_df=pd.read_csv(input_genomad_virus_summary, sep ="\t"); genomad_summary_df.columns = ['geNomad_' + str(col).replace(" " , "_") for col in genomad_summary_df.columns]

viral_list=[]
for record in SeqIO.parse(input_genomad_viral_fasta, "fasta"): viral_list.append(record.id)
geNomad_viral=pd.DataFrame(viral_list, columns=["geNomad_viral_name"]); geNomad_viral["geNomad_viral"]="Y"
viral_list_conservative=[]
for record in SeqIO.parse(input_genomad_viral_fasta_conservative, "fasta"): viral_list_conservative.append(record.id)
geNomad_viral_conservative=pd.DataFrame(viral_list_conservative, columns=["geNomad_viral_conservative_name"])
geNomad_viral_conservative["geNomad_viral_conservative"]="Y"


In [ ]:
df_temp1=vOTU_prefiltered.merge(vibrant_circular_df, left_on="vOTU_name", right_on="VIBRANT_circular_name", how="left")
df_temp2=df_temp1.merge(vibrant_positive_df, left_on="vOTU_name", right_on="VIBRANT_positive_name", how="left")
df_temp3=df_temp2.merge(vibrant_quality_df, left_on="vOTU_name", right_on="VIBRANT_scaffold", how="left")
df_temp4=df_temp3.merge(vibrant_summary_df, left_on="vOTU_name", right_on="VIBRANT_scaffold", how="left")
df_temp5=df_temp4.merge(virsorter_summary_df, left_on="vOTU_name", right_on="VirSorter2_seqname", how="left")
df_temp6=df_temp5.merge(genomad_summary_df, left_on="vOTU_name", right_on="geNomad_seq_name", how="left")
df_temp7=df_temp6.merge(geNomad_viral, left_on="vOTU_name", right_on="geNomad_viral_name", how="left")
df_temp8=df_temp7.merge(geNomad_viral_conservative, left_on="vOTU_name", right_on="geNomad_viral_conservative_name", how="left")
df_merged=df_temp8.merge(checkv_df, left_on="vOTU_name", right_on="checkV_contig_id", how="left")
df_merged=df_merged.drop(["VIBRANT_circular_name", "VIBRANT_positive_name", "VIBRANT_scaffold", "checkV_contig_id"], axis=1)
df_merged

# How many genomad, genomad conservative, virsorter, viridic
#organize so they are at the beggining with the length

In [ ]:
df_merged.to_csv(output_summary)

In [ ]:
df=df_merged[["VirSorter2_positive","VIBRANT_positive","geNomad_viral","geNomad_viral_conservative"]]

              
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

# Assuming your DataFrame is named df
# Get the counts for each category
a_count = df['VirSorter2_positive'].value_counts()['Y']
b_count = df['VIBRANT_positive'].value_counts()['Y']
c_count = df['geNomad_viral_conservative'].value_counts()['Y']
ab_count = df[df['VirSorter2_positive'] == 'Y']['VIBRANT_positive'].value_counts()['Y']
ac_count = df[df['VirSorter2_positive'] == 'Y']['geNomad_viral_conservative'].value_counts()['Y']
bc_count = df[(df['VIBRANT_positive'] == 'Y') & (df['geNomad_viral_conservative'] == 'Y')]['VIBRANT_positive'].value_counts()['Y']
abc_count = df[(df['VirSorter2_positive'] == 'Y') & (df['VIBRANT_positive'] == 'Y') & (df['geNomad_viral_conservative'] == 'Y')]['VIBRANT_positive'].value_counts()['Y']

# Create the Venn diagram
venn_labels = {'100': a_count - ab_count - ac_count + abc_count,
               '010': b_count - ab_count - bc_count + abc_count,
               '001': c_count - ac_count - bc_count + abc_count,
               '110': ab_count - abc_count,
               '101': ac_count - abc_count,
               '011': bc_count - abc_count,
               '111': abc_count}

plt.figure(figsize=(8, 6))
venn_diagram = venn3(subsets=venn_labels, set_labels=('VirSorter2_positive', 'VIBRANT_positive', 'geNomad_viral_conservative'))

# Add the counts to the Venn diagram
for text in venn_diagram.set_labels:
    text.set_fontsize(14)
for text in venn_diagram.subset_labels:
    if text:
        text.set_fontsize(12)

# Show the Venn diagram
plt.title("Venn Diagram of Categories 'VirSorter2_positive', 'VIBRANT_positive', and 'geNomad_viral_conservative'")
plt.show()


In [ ]:
df=df_merged[["VirSorter2_positive","VIBRANT_positive","geNomad_viral","geNomad_viral_conservative"]]

              
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

# Assuming your DataFrame is named df
# Get the counts for each category
a_count = df['VirSorter2_positive'].value_counts()['Y']
b_count = df['VIBRANT_positive'].value_counts()['Y']
c_count = df['geNomad_viral'].value_counts()['Y']
ab_count = df[df['VirSorter2_positive'] == 'Y']['VIBRANT_positive'].value_counts()['Y']
ac_count = df[df['VirSorter2_positive'] == 'Y']['geNomad_viral'].value_counts()['Y']
bc_count = df[(df['VIBRANT_positive'] == 'Y') & (df['geNomad_viral'] == 'Y')]['VIBRANT_positive'].value_counts()['Y']
abc_count = df[(df['VirSorter2_positive'] == 'Y') & (df['VIBRANT_positive'] == 'Y') & (df['geNomad_viral'] == 'Y')]['VIBRANT_positive'].value_counts()['Y']

# Create the Venn diagram
venn_labels = {'100': a_count - ab_count - ac_count + abc_count,
               '010': b_count - ab_count - bc_count + abc_count,
               '001': c_count - ac_count - bc_count + abc_count,
               '110': ab_count - abc_count,
               '101': ac_count - abc_count,
               '011': bc_count - abc_count,
               '111': abc_count}

plt.figure(figsize=(8, 6))
venn_diagram = venn3(subsets=venn_labels, set_labels=('VirSorter2_positive', 'VIBRANT_positive', 'geNomad_viral'))

# Add the counts to the Venn diagram
for text in venn_diagram.set_labels:
    text.set_fontsize(14)
for text in venn_diagram.subset_labels:
    if text:
        text.set_fontsize(12)

# Show the Venn diagram
plt.title("Venn Diagram of Categories 'VirSorter2_positive', 'VIBRANT_positive', and 'geNomad_viral'")
plt.show()
